In [1]:
import ccxt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#binance api를 이용해 코인 시간봉 받기
binance = ccxt.binance()

btc_ohlcv = binance.fetch_ohlcv("BTC/USDT", '1h')
df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
df.set_index('datetime', inplace=True)
df.head()

,Open,High,Low,Close,Volume
datetime,,,,,
2022-10-11 23:00:00,19039.49,19064.99,19003.25,19060.00,7124.58940
2022-10-12 00:00:00,19060.00,19132.93,19024.65,19092.17,10190.57131
2022-10-12 01:00:00,19092.17,19101.23,19030.63,19063.24,6431.61275
2022-10-12 02:00:00,19063.24,19080.61,19030.91,19066.42,5740.60223
2022-10-12 03:00:00,19065.69,19090.15,19042.61,19048.73,4711.28460


In [2]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import talib

#전략 설정
class RSI(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14


    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
    
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()

        elif crossover(self.lower_bound, self.rsi):
            self.buy()

Loading BokehJS ...

In [3]:
bt = Backtest(df, RSI, cash=50000, commission=.0015)
stats = bt.run()
stats

Start                     2022-10-11 23:00:00
End                       2022-11-01 18:00:00
Duration                     20 days 19:00:00
Exposure Time [%]                         3.2
Equity Final [$]                  52286.43168
Equity Peak [$]                   52286.47168
Return [%]                           4.572863
Buy & Hold Return [%]                 7.19638
Return (Ann.) [%]                  109.980449
Volatility (Ann.) [%]                27.75184
Sharpe Ratio                         3.962997
Sortino Ratio                             inf
Calmar Ratio                        74.842087
Max. Drawdown [%]                     -1.4695
Avg. Drawdown [%]                   -0.472357
Max. Drawdown Duration       18 days 17:00:00
Avg. Drawdown Duration        4 days 19:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       6.107746
Worst Trade [%]                      6.107746
Avg. Trade [%]                    

In [4]:
bt.plot()

Row(id='1476', ...)

In [5]:
stats = bt.optimize(upper_bound = range(55, 85, 5), 
                    lower_bound = range(10, 45, 5),
                    maximize = 'Return [%]',
                    constraint = lambda param: param.upper_bound > param.lower_bound)
stats

  0%|          | 0/14 [00:00<?, ?it/s]

Start                     2022-10-11 23:00:00
End                       2022-11-01 18:00:00
Duration                     20 days 19:00:00
Exposure Time [%]                        54.8
Equity Final [$]                  54094.88375
Equity Peak [$]                   54716.34375
Return [%]                           8.189767
Buy & Hold Return [%]                 7.19638
Return (Ann.) [%]                  269.130498
Volatility (Ann.) [%]               54.950597
Sharpe Ratio                         4.897681
Sortino Ratio                       57.657623
Calmar Ratio                       103.751486
Max. Drawdown [%]                   -2.593992
Avg. Drawdown [%]                   -0.668303
Max. Drawdown Duration        4 days 21:00:00
Avg. Drawdown Duration        0 days 22:00:00
# Trades                                    6
Win Rate [%]                        83.333333
Best Trade [%]                       4.344336
Worst Trade [%]                     -0.651431
Avg. Trade [%]                    

In [6]:
stats._strategy

<Strategy RSI(upper_bound=70,lower_bound=40)>

In [7]:
bt.plot()

Row(id='2288', ...)